In [1]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100


In [2]:
#Getting Player Info Files
# Sravan's paths
# player_info_england = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


# # Renzo's paths
player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')

# Victor's Paths

# player_info_england = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


In [3]:
# Load TM Profile Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

tm_profile_df = pd.concat([df_1, df_2])
tm_profile_df = pd.concat([tm_profile_df, df_3])

In [4]:
player_inf_lst = [player_info_england, player_info_italy, player_info_spain, player_info_france, player_info_germany]
player_info_df = pd.concat(player_inf_lst)
player_info_df.shape

(35827, 15)

In [5]:
# Cleaning repeated players

player_info_df_nodups = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_nodups))

35827
10720


In [6]:
#Loading Player Injuries

# Sravan's Path
# player_injuries_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Renzo's Path
player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Victor's Path
# player_injuries_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')


len(player_injuries_df)

55216

In [7]:
print(player_injuries_df.columns)
print(player_info_df_nodups.columns)

Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [8]:
# Load TMID to FBRefID reference table

# Sravan's Path
# fbref_to_tm_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')

# Renzo's Path
fbref_to_tm_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv')

# Victor's Path
# fbref_to_tm_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')


In [9]:
# Stripping FBRefID and TMID from other information and creating individual columns for them

# fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].apply(lambda player: player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))])

fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].str.split('/').str[5]
fbref_to_tm_df['TMID'] = fbref_to_tm_df['UrlTmarkt'].str.split('/').str[6]

fbref_to_tm_df.head()


,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back,4f974391,92571
2,Aarón Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper,67669ce7,284430
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield,565c3fe4,276566
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back,1780bb4a,591949


In [10]:
# converting TMId to string in player_injuries_df

player_injuries_df['TMId'] = player_injuries_df['TMId'].astype(str)
# player_injuries_df.head()

In [11]:
# Merging on intersection of player_injuries_df and fbref_to_tm_df on columns TMId and TMID respectively

player_injuries_df_2 = pd.merge(left=player_injuries_df, right=fbref_to_tm_df, left_on='TMId', right_on='TMID', how='inner')

# player_injuries_df_2.head()

In [12]:
# Determining number of unique player in our data

len(player_injuries_df_2['name'].unique())

4614

In [13]:
# removing duplicates from player_info_df

player_info_df = player_info_df.drop_duplicates()

# player_info_df.head()

In [14]:
# Merging Player Injuries with FBRef Profiles

player_injuries_info_df = pd.merge(left=player_injuries_df_2, right=player_info_df, left_on='FBRefID', right_on='FBRefId', how='inner')

len(player_injuries_info_df)

32584

In [15]:
# finding number of unique players
 
len(player_injuries_info_df['name'].unique())

4586

In [16]:
# Merge with TM Profile information

tm_profile_df['TMId'] = tm_profile_df['TMId'].astype(str)
player_injuries_profile_final = pd.merge(left=player_injuries_info_df, right=tm_profile_df, left_on='TMId', right_on='TMId', how='inner')
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermark

In [17]:
# Creating new columns with features

player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['from'] != '-']
player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['until'] != '-']
player_injuries_profile_final['injury_year'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').year)
player_injuries_profile_final['injury_week'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['release_week'] = player_injuries_profile_final['until'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['from'] = pd.to_datetime(player_injuries_profile_final['from'])
player_injuries_profile_final['until'] = pd.to_datetime(player_injuries_profile_final['until'])

In [18]:
# Exploding Dataframe to weekly basis

# player_injuries_profile_final['from']=player_injuries_profile_final.apply(lambda x : pd.date_range(start=x['from'], end=x['until'], freq='W') ,axis=1)
# player_injuries_profile_final = player_injuries_profile_final.explode('from')
# player_injuries_profile_final['until']=player_injuries_profile_final['from']+pd.tseries.offsets.MonthEnd()

# player_injuries_profile_final.head(50)

def ran_week(row):
    return list(pd.date_range(row['from'], row['until'], freq='w'))

# def ran_year(row):
    # return list(pd.date_range(row['from'], row['until'], freq='y'))

player_injuries_profile_final['injury_week'] = player_injuries_profile_final['injury_week'].astype(int)
player_injuries_profile_final['release_week'] = player_injuries_profile_final['release_week'].astype(int)
player_injuries_profile_final['current_week'] = player_injuries_profile_final.apply(ran_week, axis = 1)
# player_injuries_profile_final['current_year'] = player_injuries_profile_final.apply(ran_year, axis = 1)

player_injuries_profile_final = player_injuries_profile_final.explode('current_week')

In [19]:
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_week
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-09-30
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-07
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-14
0,18/19,Hamstring Injury,2018-09-29,2018-10-27,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,2018-10-21
1,16/17,Calf Strain,2017-03-18,2017-04-

In [20]:
# player_injuries_profile_final[player_injuries_profile_final['name_x'] == 'Petr Cech'].tail(60)

In [21]:
player_injuries_profile_final['current_year'] = player_injuries_profile_final['current_week'].apply(lambda x: x.year)#strftime('%W'))
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_week'].apply(lambda x: x.week)

In [22]:
player_injuries_profile_final['current_week'] = player_injuries_profile_final['current_week'].fillna(player_injuries_profile_final['injury_week'])
player_injuries_profile_final['current_year'] = player_injuries_profile_final['current_year'].fillna(player_injuries_profile_final['injury_year'])

In [23]:
# player_injuries_profile_final[player_injuries_profile_final['name_x'] == 'Petr Cech'].tail(60)

In [24]:
# Saving player_injuries_profile_final to Google Drive

# Renzo's path

player_injuries_profile_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_profile_final.csv', index=False)

In [48]:
# Loading Match Dataframes

# Renzo's Path
total_match_logs_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

# Sravan's Path
# total_match_logs_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

# Victor's Path
# total_match_logs_df = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [50]:
# Creating 'week' and 'year' column in total_match_logs_df

total_match_logs_df['week'] = pd.to_datetime(total_match_logs_df['Date']).dt.week
total_match_logs_df['year'] = pd.to_datetime(total_match_logs_df['Date']).dt.year


<ipython-input-50-c9fe56b8ae89>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  total_match_logs_df['week'] = pd.to_datetime(total_match_logs_df['Date']).dt.week


In [51]:
print(len(total_match_logs_df.columns))
print(len(player_injuries_profile_final.columns))

49
74


In [52]:
# Re-engineer Columns
# Results
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'W', 'Won'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'W', 'Won'] = 0

total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'L', 'Loss'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'L', 'Loss'] = 0

total_match_logs_df.loc[total_match_logs_df['Result'].str[0] == 'D', 'Draw'] = 1
total_match_logs_df.loc[total_match_logs_df['Result'].str[0] != 'D', 'Draw'] = 0

# Starts
total_match_logs_df.loc[total_match_logs_df['Start'] == 'Y', 'Games_Start'] = 1
total_match_logs_df.loc[total_match_logs_df['Start'] != 'Y', 'Games_Start'] = 0

total_match_logs_df.head()

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,week,year,Won,Loss,Draw,Games_Start
0,2014-08-24,Sun,La Liga,Matchweek 1,Away,W 2–0,Villarreal,Levante,N,NaN,23,0,1,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,2014,1.0,0.0,0.0,0.0
1,2014-08-28,Thu,Europa Lg,Play-off round,Home,W 4–0,es Villarreal,kz FC Astana,Y,NaN,62,0,1,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2014,1.0,0.0,0.0,1.0
2,2014-08-31,Sun,La Liga,Matchweek 2,Home,L 0–1,Villarreal,Barcelona,N,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2014,0.0,1.0,0.0,0.0
3,2014-09-14,Sun,La Liga,Matchweek 3,Away,D 0–0,Villarreal,Granada,Y,NaN,90,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2014,0.0,0.0,1.0,1.0
4,2014-09-18,Thu,Europa Lg,Group stage,Away,D 1–1,es Villarreal,de M'Gladbach,Y,NaN,90,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moi-Gomez,5a626448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,2014,0.0,0.0,1.0,1.0


In [53]:
total_match_logs_df = total_match_logs_df.replace('On matchday squad, but did not play', 0)
total_match_logs_df['Min'] = total_match_logs_df['Min'].astype(float)
total_match_logs_df['Gls'] = total_match_logs_df['Gls'].astype(float)
total_match_logs_df['Ast'] = total_match_logs_df['Ast'].astype(float)
total_match_logs_df['PK'] = total_match_logs_df['PK'].astype(float)
total_match_logs_df['PKatt'] = total_match_logs_df['PKatt'].astype(float)
total_match_logs_df['Sh'] = total_match_logs_df['Sh'].astype(float)
total_match_logs_df['SoT'] = total_match_logs_df['SoT'].astype(float)
total_match_logs_df['Att.1'] = total_match_logs_df['Att.1'].astype(float)
total_match_logs_df['Fls'] = total_match_logs_df['Fls'].astype(float)
total_match_logs_df['Fld'] = total_match_logs_df['Fld'].astype(float)
total_match_logs_df['Off'] = total_match_logs_df['Off'].astype(float)
total_match_logs_df['Crs'] = total_match_logs_df['Crs'].astype(float)
total_match_logs_df['TklW'] = total_match_logs_df['TklW'].astype(float)
total_match_logs_df['OG'] = total_match_logs_df['OG'].astype(float)
total_match_logs_df['PKwon'] = total_match_logs_df['PKwon'].astype(float)
total_match_logs_df['PKcon'] = total_match_logs_df['PKcon'].astype(float)
total_match_logs_df['CrdY'] = total_match_logs_df['CrdY'].astype(float)
total_match_logs_df['CrdR'] = total_match_logs_df['CrdR'].astype(float)
total_match_logs_df['Touches'] = total_match_logs_df['Touches'].astype(float)
total_match_logs_df['Press'] = total_match_logs_df['Press'].astype(float)
total_match_logs_df['Tkl'] = total_match_logs_df['Tkl'].astype(float)
total_match_logs_df['Int'] = total_match_logs_df['Int'].astype(float)
total_match_logs_df['Blocks'] = total_match_logs_df['Blocks'].astype(float)
total_match_logs_df['xG'] = total_match_logs_df['xG'].astype(float)
total_match_logs_df['npxG'] = total_match_logs_df['npxG'].astype(float)
total_match_logs_df['xA'] = total_match_logs_df['xA'].astype(float)
total_match_logs_df['SCA'] = total_match_logs_df['SCA'].astype(float)
total_match_logs_df['GCA'] = total_match_logs_df['GCA'].astype(float)
total_match_logs_df['Cmp'] = total_match_logs_df['Cmp'].astype(float)
total_match_logs_df['Att'] = total_match_logs_df['Att'].astype(float)
total_match_logs_df['Prog'] = total_match_logs_df['Prog'].astype(float)
total_match_logs_df['Carries'] = total_match_logs_df['Carries'].astype(float)
total_match_logs_df['Prog.1'] = total_match_logs_df['Prog.1'].astype(float)
total_match_logs_df['Succ'] = total_match_logs_df['Succ'].astype(float)



In [54]:
total_match_logs_df_05 = total_match_logs_df.drop_duplicates()

In [55]:
total_match_logs_df_05[(total_match_logs_df_05['year'] == 2006) & (total_match_logs_df_05['FBRefID'] == '71672fa0') & (total_match_logs_df_05['week'] == 41)].sort_values('Date')
# total_match_logs_df_05[(total_match_logs_df_05['FBRefID'] == '71672fa0')].sort_values('Date')

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,week,year,Won,Loss,Draw,Games_Start
34453,2006-10-11,Wed,Euro Qualifying,Group stage,Away,D 1–1,cz Czech Republic,ie Republic of Ireland,Y,NaN,90.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,41,2006,0.0,0.0,1.0,1.0
34454,2006-10-14,Sat,Premier League,Matchweek 8,Away,W 1–0,Chelsea,Reading,Y,NaN,4.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,41,2006,1.0,0.0,0.0,1.0


In [57]:
# Eliminated lowercase component of 'Squad' column

def no_lowercase(s):
    lst = []
    for i in s.split(' '):
        if i.islower():
            continue
        else:
            lst.append(i)
    
    return ' '.join(lst)
        # no_lc = ' '.join([word for word in i.split(' ') if not word.islower()])

total_match_logs_df_05['Squad'] = total_match_logs_df_05['Squad'].apply(no_lowercase)


<ipython-input-57-4d58e83e1db8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_match_logs_df_05['Squad'] = total_match_logs_df_05['Squad'].apply(no_lowercase)


In [64]:
# Sanity Checks

# total_match_logs_df_05[(total_match_logs_df_05['year'] == 2006) & (total_match_logs_df_05['FBRefID'] == '71672fa0')].sort_values('Date')
# dict(total_match_logs_df_05['Squad'].value_counts()[1000:-1])

In [65]:
# Choosing wanted columns

match_log_columns = ['Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
 'SCA', 'GCA', 'Cmp', 'Att', 'Prog', 'Carries', 'Prog.1', 'Succ',
 'Att.1', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
 'PKwon', 'PKcon', 'week', 'year', 'Won', 'Loss', 'Draw']

total_match_logs_df_2 = total_match_logs_df_05[match_log_columns]

# total_match_logs_df_2 = total_match_logs_df_2.replace('On matchday squad, but did not play', 0)
# total_match_logs_df_2['Min'] = total_match_logs_df_2['Min'].astype(float)
# total_match_logs_df_2['Gls'] = total_match_logs_df_2['Gls'].astype(float)
# total_match_logs_df_2['Ast'] = total_match_logs_df_2['Ast'].astype(float)
# total_match_logs_df_2['PK'] = total_match_logs_df_2['PK'].astype(float)
# total_match_logs_df_2['PKatt'] = total_match_logs_df_2['PKatt'].astype(float)
# total_match_logs_df_2['Sh'] = total_match_logs_df_2['Sh'].astype(float)
# total_match_logs_df_2['SoT'] = total_match_logs_df_2['SoT'].astype(float)
# total_match_logs_df_2['Att.1'] = total_match_logs_df_2['Att.1'].astype(float)
# total_match_logs_df_2['Fls'] = total_match_logs_df_2['Fls'].astype(float)
# total_match_logs_df_2['Fld'] = total_match_logs_df_2['Fld'].astype(float)
# total_match_logs_df_2['Off'] = total_match_logs_df_2['Off'].astype(float)
# total_match_logs_df_2['Crs'] = total_match_logs_df_2['Crs'].astype(float)
# total_match_logs_df_2['TklW'] = total_match_logs_df_2['TklW'].astype(float)
# total_match_logs_df_2['OG'] = total_match_logs_df_2['OG'].astype(float)
# total_match_logs_df_2['PKwon'] = total_match_logs_df_2['PKwon'].astype(float)
# total_match_logs_df_2['PKcon'] = total_match_logs_df_2['PKcon'].astype(float)
# total_match_logs_df_2['CrdY'] = total_match_logs_df_2['CrdY'].astype(float)
# total_match_logs_df_2['CrdR'] = total_match_logs_df_2['CrdR'].astype(float)
# total_match_logs_df_2['Touches'] = total_match_logs_df_2['Touches'].astype(float)
# total_match_logs_df_2['Press'] = total_match_logs_df_2['Press'].astype(float)
# total_match_logs_df_2['Tkl'] = total_match_logs_df_2['Tkl'].astype(float)
# total_match_logs_df_2['Int'] = total_match_logs_df_2['Int'].astype(float)
# total_match_logs_df_2['Blocks'] = total_match_logs_df_2['Blocks'].astype(float)
# total_match_logs_df_2['xG'] = total_match_logs_df_2['xG'].astype(float)
# total_match_logs_df_2['npxG'] = total_match_logs_df_2['npxG'].astype(float)
# total_match_logs_df_2['xA'] = total_match_logs_df_2['xA'].astype(float)
# total_match_logs_df_2['SCA'] = total_match_logs_df_2['SCA'].astype(float)
# total_match_logs_df_2['GCA'] = total_match_logs_df_2['GCA'].astype(float)
# total_match_logs_df_2['Cmp'] = total_match_logs_df_2['Cmp'].astype(float)
# total_match_logs_df_2['Att'] = total_match_logs_df_2['Att'].astype(float)
# total_match_logs_df_2['Prog'] = total_match_logs_df_2['Prog'].astype(float)
# total_match_logs_df_2['Carries'] = total_match_logs_df_2['Carries'].astype(float)
# total_match_logs_df_2['Prog.1'] = total_match_logs_df_2['Prog.1'].astype(float)
# total_match_logs_df_2['Succ'] = total_match_logs_df_2['Succ'].astype(float)

In [66]:
# Sanity check of total_match_logs_df_2

# total_match_logs_df_2[(total_match_logs_df_2['year'] == 2006) & (total_match_logs_df_2['FBRefID'] == '71672fa0')].sort_values('week')

,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,name,FBRefID,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,week,year,Won,Loss,Draw
34422,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,2006,1.0,0.0,0.0
34423,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2,2006,1.0,0.0,0.0
34424,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3,2006,0.0,0.0,1.0
34425,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5,2006,0.0,0.0,1.0
34426,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,5,2006,1.0,0.0,0.0
34427,90.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6,2006,0.0,1.0,0.0
34428,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,8,2006,0.0,1.0,0.0
34429,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,8,2006,1.0,0.0,0.0
34430,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,9,2006,1.0,0.0,0.0
34431,90.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Petr-Cech,71672fa0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,10,2006,0.0,0.0,1.0


In [67]:
# Grouping total_match_logs_df_2 by name, FBRefID, week and year

total_match_logs_df_3 = total_match_logs_df_2.groupby(by=['name', 'FBRefID','week', 'year']).sum().reset_index()

In [68]:
# Sanity checks for total_match_logs_df_3

total_match_logs_df_3[(total_match_logs_df_3['FBRefID'] == '71672fa0') & (total_match_logs_df_3['year'] == 2006)].sort_values(['year', 'week'])

,name,FBRefID,week,year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw
1192938,Petr-Cech,71672fa0,1,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1192950,Petr-Cech,71672fa0,2,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1192966,Petr-Cech,71672fa0,3,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1192987,Petr-Cech,71672fa0,5,2006,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1193004,Petr-Cech,71672fa0,6,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1193026,Petr-Cech,71672fa0,8,2006,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1193042,Petr-Cech,71672fa0,9,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1193057,Petr-Cech,71672fa0,10,2006,180.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1193070,Petr-Cech,71672fa0,11,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1193084,Petr-Cech,71672fa0,12,2006,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [69]:
# Merging total_match_logs_df with player_injuries_profile_final

complete_final_df = pd.merge(left=total_match_logs_df_3, right=player_injuries_profile_final, left_on=['week', 'year', 'FBRefID'], right_on=['current_week', 'current_year','FBRefID'], how='outer')

In [70]:
# Cleaning complete_final_df

complete_final_df.loc[complete_final_df['week'].isna(), 'agg_week'] = complete_final_df['current_week']
complete_final_df.loc[complete_final_df['year'].isna(), 'agg_year'] = complete_final_df['current_year']
complete_final_df.loc[complete_final_df['current_week'].isna(), 'agg_week'] = complete_final_df['week']
complete_final_df.loc[complete_final_df['current_year'].isna(), 'agg_year'] = complete_final_df['year']
complete_final_df.loc[complete_final_df['agg_week'].isna(), 'agg_week'] = complete_final_df['week']
complete_final_df.loc[complete_final_df['agg_year'].isna(), 'agg_year'] = complete_final_df['year']


In [71]:
# Sorting complete_final_df

pd.set_option("display.max_columns", None)
complete_final_df.sort_values(['year', 'week'], inplace=True)

In [72]:
# Sanity Checks complete_final_df

complete_final_df[complete_final_df['FBRefId'] == '71672fa0']

,name,FBRefID,week,year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,current_week,current_year,agg_week,agg_year
1193484,Petr-Cech,71672fa0,45.0,2007.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,07/08,Fibula Fracture,2007-11-08,2007-12-10,32 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007.0,45.0,50.0,45.0,2007.0,45.0,2007.0
1193578,Petr-Cech,71672fa0,52.0,2007.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,07/08,Hip problems,2007-12-24,2007-12-24,0 days,-,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007.0,52.0,52.0,52.0,2007.0,52.0,2007.0
1193071,Petr-Cech,71672fa0,6.0,2013.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12/13,Fractured Finger,2013-02-04,2013-02-07,3 days,-,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,6.0,6.0,6.0,2013.0,6.0,2013.0
1193220,Petr-Cech,716

In [97]:
total_match_logs_df_3

,Squad,name,FBRefID,week,year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw
0,(MG),Ceara,e1d326d9,34,2017,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,(MG),Ceara,e1d326d9,36,2017,90.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,(MG),Ceara,e1d326d9,37,2017,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,(MG),Ceara,e1d326d9,38,2017,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,(MG),Ceara,e1d326d9,39,2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536813,Žilina,Peter-Pekarik,c18d22f5,40,2008,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1536814,Žilina,Peter-Pekarik,c18d22f5,43,2008,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1536815,Žilina,Peter-Pekarik,c18d22f5,45,2008,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1536816,Žilina,Peter-Pekarik,c18d22f5,48,2008,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
